# Part 2: Bayesian Optimization

In the second part of the tutorial, we are going to use the GPy library and Gaussian process regression to optimize a black-box function

In [ ]:
import GPy
import numpy as np
import matplotlib.pyplot as plt

As a simple benchmark problem, we use a small mockup machine interface.

In [ ]:
class Accelerator:
    def __init__(self,sigma = 1.0):
        self.x = np.array([0.1, 0.2])
        self.sigma = sigma
        
    def get(self, **kwargs):
        """
        Returns the signal measurement.
        
        Examples:
            get()  # returns a measurement with the current parameter settings
            get(x=[0.1,0.4])  # sets the parameters x1=0.1, x2=0.4, then returns a measurement
            get(x1=0.2)  # sets x1=0.2 and keeps the second parameter fixed
            get(x1=0.1, x2=0.4)  # sets x1=0.1, x2=0.4
        
        Optional parameters:
            x: array-like 2d
            x1: scalar
            x2: scalar
        """
        if 'x' in kwargs:
            self.x = np.array(kwargs['x'])
        else:
            for i in range(len(self.x)):
                self.x[i] = kwargs.get(f'x{i+1}', self.x[i])
        
        xx, yy = self.x[0], self.x[1]
        y = (4. - 2.1*xx**2 + (xx**4)/3.)*(xx**2) + xx*yy + (-4. + 4*(yy**2))*(yy**2)
        return np.maximum(-y + 2.5, 0) + np.sqrt(self.sigma)*np.random.normal(0,1)


def plot_1d_gp(gp, xlim, 
               plot_data=True, 
               plot_confidence=False, 
               s=2,
               axis=None, color='C0'):
    """
    Plots a 1d gp.
    """
    
    if axis is None:
        axis = plt.gca()
        
    # define an evaluation set
    X_eval = np.linspace(*xlim, 100).reshape(-1,1)  # GPy expects shape (num_points, input_dim)
    
    # compute posterior mean and variance
    Y_pred, Y_var = gp.predict_noiseless(X_eval)
    _, Y_noise_var = gp.predict(X_eval)
    
    # flatten the arrays for pyplot
    X_eval = X_eval.reshape(-1)
    Y_pred = Y_pred.reshape(-1)
    Y_var = Y_var.reshape(-1)
    Y_noise_var = Y_noise_var.reshape(-1)

    # plot mean
    axis.plot(X_eval, Y_pred, zorder=5, color=color, label = "GP mean")
    
    # plot data if it's not the 'fake' data point
    if plot_data and not (gp.X.shape[0] == 1 and gp.X[0] > 0.9*10e10):
        axis.scatter(gp.X, gp.Y, zorder=10, marker='x', color=color, label = "data")
        
    # plot the predictive uncertainty interval
    if plot_confidence:
        axis.fill_between(X_eval, Y_pred-s*np.sqrt(Y_var), Y_pred+s*np.sqrt(Y_var), alpha=0.4, color=color)

    axis.set_xlim(xlim)  # set x-axis limits



## a) Grid optimizer - 1d


In [ ]:
noise_level = 0.1 ## std of noise level
accelerator_no_noise = Accelerator(sigma = 0.0) # accelerator without noise
accelerator_noise = Accelerator(sigma = noise_level) # accelerator with realistic noise

n = 1000 # number of grid points (fine)
X_orig = X = np.linspace(-2,2,n) # fine grid of the parameter domain

no_grid_points = 10 # initial small grid 
grid = np.linspace(-2,2,no_grid_points) # initial small grid

Grid with no noise fails due to efficiency even with noisless oracle: $y = f(x)$

In [ ]:
Y_no_noise = np.array([[accelerator_no_noise.get(x1=x)] for x in X]) # Y data
grid_values = np.array([[accelerator_no_noise.get(x1=x)] for x in grid]) # X data

# plot the noiseless grid
plt.plot(X,Y_no_noise,"r-",alpha = 0.7)
plt.plot(grid,grid_values,'bo')

print ("Maximum achieved: ", np.max(grid_values), "Optimality gap:", np.max(Y_no_noise) - np.max(grid_values))

Having no noise is unrealistic, hence we add observational noise: $y = f(x) + \epsilon$

In [ ]:
grid_values = np.array([[accelerator_noise.get(x1=x)] for x in grid])
plt.plot(X,Y_no_noise,alpha = 0.7, lw = 3, color= 'red', label = "true function")
plt.legend()

In [ ]:
plt.plot(X,Y_no_noise,alpha = 0.7, lw = 3, color = "red", label = "true function")
plt.plot(grid,grid_values,'bx', label = "grid - 10 points")
plt.legend()

## b) Initial data and confidence bounds - 1d

In this example, we will see how probabilistic model (GP) allows us to restrict the search space using high probabilistic confidence estimates


In [ ]:
plt.ylim(-3,5)
plt.xlim(-2,2)
plt.plot(X,Y_no_noise,alpha = 0.7, lw = 3, color = "red", label = "true function")
plt.legend()

In [ ]:
# small initial grid grid 
no_small_grid_points = 5
grid_small = np.linspace(-2,2,no_small_grid_points)
grid_small_value = np.array([[accelerator_noise.get(x1=x)] for x in grid_small])
grid_small = grid_small.reshape(-1,1)

# Define a kernel
rbf = GPy.kern.RBF(input_dim=1, lengthscale=0.5, variance=5)
# Define the GP object
gp = GPy.models.GPRegression(X=grid_small,
                             Y=grid_small_value,
                             kernel=rbf, noise_var=noise_level)

In [ ]:
plt.ylim(-3,5)
plt.xlim(-2,2)
plot_1d_gp(gp, xlim=(-2,2),
            plot_data = True,
            plot_confidence = True)
plt.plot(X_orig,Y_no_noise,alpha = 0.7, lw = 3, color = "red", label = "true function")
plt.legend()

In [ ]:
plt.ylim(-3,5)
plt.xlim(-2,2)
plot_1d_gp(gp, xlim=(-2,2),
            plot_data = True,
            plot_confidence = True)
# get mean and variance from the gp model
Y_mean, Y_var = gp.predict_noiseless(X.reshape(-1,1))
# plot the lower confidence bound
plt.plot(X_orig,X_orig*0+np.max(Y_mean-2*np.sqrt(Y_var)),'--',color = "orange")
plt.plot(X_orig,Y_no_noise,alpha = 0.7, lw = 3, color = "red", label = "true function")
plt.legend()

In [ ]:
limit = np.max(Y_mean-2*np.sqrt(Y_var)) # maximum of lower confidence estimate (95%)
ucb = Y_mean+2*np.sqrt(Y_var) # upper confidence estimate (95%)
discarted = X_orig[(ucb<limit)[:,0]]

In [ ]:
plt.ylim(-3,5)
plt.xlim(-2,2)
plot_1d_gp(gp, xlim=(-2,2),
            plot_data = True,
            plot_confidence = True)
plt.plot(X_orig,X_orig*0+np.max(Y_mean-2*np.sqrt(Y_var)),'--',color = "orange")
plt.plot(X_orig,Y_no_noise,alpha = 0.7, lw = 3, color = "red", label = "true function")
plt.ylim(-3,5)
plt.xlim(-2,2)
i = 0
for x in X_orig:
    if (ucb<limit)[i,0]:
        plt.fill_between([x], -3,5, alpha=0.4, color='orange')
    i=i+1
plt.legend()

## c) UCB algorithm - 1d


In [ ]:
def UCB_step(X,Y,beta,kernel,discretization):

    """
    Returns an optimizer of a UCB acquisiton function given a GP model with kernel and data (X,Y). 
    Discretization determines the optimization space for the acquisiton function    
    """  
    
    # Define the GP object
    gp = GPy.models.GPRegression(X=X,
                                 Y=Y,
                             kernel=kernel, noise_var=noise_level)
    
    # predict mean and variance
    Y_mean, Y_var = gp.predict_noiseless(discretization)

    # construct acquisition function
    acquisition_function = Y_mean + beta * np.sqrt(Y_var)
    
    # maximize it to get candidate point
    candidate_point = discretization[np.argmax(acquisition_function)]
 
    return (candidate_point,gp)

def plot_acquisition(gp,discretization):
    # predict mean and variance
    Y_mean, Y_var = gp.predict_noiseless(discretization.reshape(-1,1))

    # construct acquisition function
    acquisition_function = Y_mean + beta * np.sqrt(Y_var)
    
    # maximize it to get candidate point
    candidate_point = discretization[np.argmax(acquisition_function)]
    
    # plot
    plt.plot(discretization, acquisition_function,'g',label = "UCB acquisition")
    plt.plot(candidate_point, np.max(acquisition_function), marker = 's', color = 'g') 


In [ ]:
# we reuse previous small grid as initial dataset for our Bayesian Optimization Algorithm
X = grid_small
Y = grid_small_value

The next section defines design choices as kernel and the exploration/exploitation parameter. You can change and play with the settings to see the effect and convergence

In [ ]:
# Define a kernel
kernel = GPy.kern.RBF(input_dim=1, lengthscale=0.5, variance=5)

# Define discretization of the domain
discretization = np.linspace(-2,2,1000).reshape(-1,1)

# We define the exploration/exploitation parameter according to a common heuristic
beta = 2.

The next code implements one step of GP-UCB algorithm. When rerunning the cells next step of the BO algorithm is performed and the current posterior of GP is plotted

In [ ]:
# Define a kernel
kernel = GPy.kern.RBF(input_dim=1, lengthscale=0.5, variance=5)

# Define discretization of the domain
discretization = np.linspace(-2,2,1000).reshape(-1,1)

# We define the exploration/exploitation parameter according to a common heuristic
beta = 2.

# acquisiton of the next data point
x_new, gp = UCB_step(X,Y,beta,kernel,discretization) # make a UCB step

# evaluation of the new data point
y_new = accelerator_noise.get(x1 = x_new)

# appending the new data point
X = np.append(X,x_new.reshape(-1,1), axis = 0)
Y = np.append(Y,y_new.reshape(-1,1), axis =0)

# plot everything
plot_1d_gp(gp, xlim=(-2,2),
            plot_data = True,
            plot_confidence = True)
plot_acquisition(gp,discretization)
plt.plot(X_orig,Y_no_noise,label = "true function", color = "r")
plt.legend()


In [ ]:
def GP_UCB_optimize(budget,kernel,beta,X_init,Y_init, discretization, output = None):
    """
    Implements GP-UCB algorithm
    
    budget (int) : number of steps to be taken
    kernel (GPY object): kernel type
    beta (float): exploration/exploitation parameter
    X_init(numpy 2D) : initial data
    Y_init(numpy 2D) : initial data
    discretization (numpy 2D): discretization array for aquisition function optimization
    
    """
    X = X_init
    Y = Y_init
    
    d = X_init.shape[1]
    
    for t in range(budget):

        # acquisiton of the next data point
        x_new, gp = UCB_step(X,Y,beta,kernel,discretization) # make a UCB step
    
        # evaluation of the new data point
        if d > 1:
            y_new = accelerator_noise.get(x1 = x_new[0], x2 = x_new[1])
        else:
            y_new = accelerator_noise.get(x1 = x_new)

        # appending the new data point
        X = np.append(X,x_new.reshape(-1,d), axis = 0)
        Y = np.append(Y,y_new.reshape(-1,1), axis = 0)
        
    Y_mean, Y_var = gp.predict_noiseless(discretization)
    best_point = discretization[np.argmax(Y_mean)]
    if output == "full":
        return best_point, gp, X, Y
    else:
        return best_point, gp

In the following experiment we will use the same number of point in order to optimize the function as the grid search:

In [ ]:
# run the optimization 
optimal_point, gp = GP_UCB_optimize(7,kernel,3.0,grid_small,grid_small_value,discretization)

# determining the location and value of optimal point
index = np.abs(X_orig-optimal_point).argmin()
optimal_value = Y_no_noise[index]
print ("Optimal point: ", optimal_point)
print ("Maximum achieved: ",  optimal_value, "Optimality gap: ", np.max(Y_no_noise) -  optimal_value)

# final plot

# plot everything
plot_1d_gp(gp, xlim=(-2,2),
            plot_data = True,
            plot_confidence = True)
plot_acquisition(gp,discretization)
plt.plot(X_orig,Y_no_noise,label = "true function", color = "r")
plt.legend()

## e) 2d example

In [ ]:
# Define a kernel
kernel_2d = GPy.kern.RBF(input_dim=2, lengthscale=0.5, variance=5) # note the change to 2 dimensions

# Define discretization of the domain (2D)
x = np.linspace(-2,2,50)
y = np.linspace(-1,1,50) # note the second parameter varries over different domain
discretization_2d = np.transpose([np.tile(x, len(y)), np.repeat(y, len(x))])

# We define the exploration/exploitation parameter according to a common heuristic
beta = 2.

# initial data
N = 3 # initial number of points

X_2d = np.random.uniform(low = -1, high = 1, size = (N,2))
Y_2d = np.array([[accelerator_noise.get(x=x)] for x in X_2d])

# running the method
T = 10
optimal_point, gp, X, Y = GP_UCB_optimize(T,kernel_2d,2.0,X_2d,Y_2d,discretization_2d, output = "full")

print ("Optimal point: ", optimal_point)

In [ ]:
# Final plot
from scipy.interpolate import griddata

# predict mean and var:
Y_mean, _  = gp.predict_noiseless(discretization_2d)
Y_mean_max = gp.predict_noiseless(np.array([optimal_point]))

plt.clf()
ax = plt.axes(projection='3d')
xx = discretization_2d[:, 0]
yy = discretization_2d[:, 1]
grid_x, grid_y = np.mgrid[min(xx):max(xx):100j, min(yy):max(yy):100j]
grid_z_mu = griddata((xx, yy), Y_mean[:, 0], (grid_x, grid_y), method='linear')
ax.scatter(X[:, 0], X[:, 1], Y[:,0], c='r', s=100, marker="o", depthshade=False)
ax.scatter(optimal_point[0],optimal_point[1], Y_mean_max[0], c='b', s=100, marker="o", depthshade=False)
ax.plot_surface(grid_x, grid_y, grid_z_mu, color='r', alpha=0.4)
plt.title('Posterior mean prediction')
plt.show()